In [1]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import keras
from PIL import Image
# from IPython.display import display, HTML
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense
from keras import regularizers
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder # one-hot encoding for age
from keras.utils.np_utils import to_categorical


import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os
import numpy as np 
import pandas as pd
import os, sys
from time import time
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers, callbacks
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Convolution2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras import models, layers
import glob

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [3]:
df['Id'] = df['Id'] + '.jpg';
df['Id']

0       0007de18844b0dbbb5e1f607da0606e0.jpg
1       0009c66b9439883ba2750fb825e1d7db.jpg
2       0013fd999caf9a3efe1352ca1b0d937e.jpg
3       0018df346ac9c1d8413cfcc888ca8246.jpg
4       001dc955e10590d3ca4673f034feeef2.jpg
                        ...                 
9907    ffbfa0383c34dc513c95560d6e1fdb57.jpg
9908    ffcc8532d76436fc79e50eb2e5238e45.jpg
9909    ffdf2e8673a1da6fb80342fa3b119a20.jpg
9910    fff19e2ce11718548fa1c5d039a5192a.jpg
9911    fff8e47c766799c9e12f3cb3d66ad228.jpg
Name: Id, Length: 9912, dtype: object

In [4]:
df_training = df

In [5]:
df_training.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0.jpg,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db.jpg,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e.jpg,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246.jpg,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2.jpg,0,0,0,1,0,0,1,0,0,0,0,0,72


In [7]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_training) * TRAIN_PCT)

df_train = df_training[0:TRAIN_CUT]
df_validate = df_training[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 8920
Validate size: 992


In [9]:
training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="train/",
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        batch_size=32,
        class_mode='raw')

Found 8920 validated image filenames.


In [11]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory="train/",
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        class_mode='raw')

Found 992 validated image filenames.


In [12]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

lr_schedule = ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=100, decay_rate=0.96,
        staircase=True)

In [13]:
def create_scnn():
  smodel = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  return smodel

In [14]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1
		# define the model input
	inputs = Input(shape=inputShape)
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs
		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
	# construct the CNN
	model = Model(inputs, x)
	# return the CNN
	return model

In [15]:
model = create_scnn()

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 246016)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               125960704 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5

In [17]:
#model.compile(loss = 'mean_squared_error', optimizer='adam')
model.compile(loss = 'mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule))

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  verbose = 1, 
  validation_data=val_generator, callbacks=[monitor], epochs=25)

Epoch 1/25
279/279 [==============================] - 953s 3s/step - loss: 589.7477 - val_loss: 447.0041
Epoch 2/25
279/279 [==============================] - 932s 3s/step - loss: 444.7310 - val_loss: 416.3904
Epoch 3/25
279/279 [==============================] - 1029s 4s/step - loss: 442.2500 - val_loss: 420.2144
Epoch 4/25
279/279 [==============================] - 1069s 4s/step - loss: 438.0660 - val_loss: 452.3085
Epoch 5/25
279/279 [==============================] - 1026s 4s/step - loss: 427.9590 - val_loss: 418.8467
Epoch 6/25
279/279 [==============================] - 1078s 4s/step - loss: 428.3715 - val_loss: 420.1753
Epoch 7/25
279/279 [==============================] - 954s 3s/step - loss: 416.4424 - val_loss: 413.7735
Epoch 8/25
279/279 [==============================] - 926s 3s/step - loss: 412.3568 - val_loss: 419.2469
Epoch 9/25
279/279 [==============================] - 927s 3s/step - loss: 404.1393 - val_loss: 415.1210
Epoch 10/25
279/279 [==============================

In [18]:
# https://github.com/sahupra1357/Pawpularity/blob/main/PetFinderCNNs.ipynb

In [30]:
test_df = pd.read_csv('test.csv')
test_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0


In [31]:
Ids = test_df["Id"]

test_df['Id'] = test_df['Id'] + '.jpg';
test_df['Id']

0    4128bae22183829d2b5fea10effdb0c3.jpg
1    43a2262d7738e3d420d453815151079e.jpg
2    4e429cead1848a298432a0acad014c9d.jpg
3    80bc3ccafcc51b66303c2c263aa38486.jpg
4    8f49844c382931444e68dffbe20228f4.jpg
5    b03f7041962238a7c9d6537e22f9b017.jpg
6    c978013571258ed6d4637f6e8cc9d6a3.jpg
7    e0de453c1bffc20c22b072b34b54e50f.jpg
Name: Id, dtype: object

In [32]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="test/",
        x_col="Id",
        batch_size=1,
        shuffle=False,
        target_size=(256, 256),
        class_mode=None)

Found 8 validated image filenames.


In [33]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(test_df))

In [34]:
df_submit = pd.DataFrame({'Id':Ids,'Pawpularity':pred.flatten()})
df_submit

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3.jpg,32.349209
1,43a2262d7738e3d420d453815151079e.jpg,36.716797
2,4e429cead1848a298432a0acad014c9d.jpg,35.214756
3,80bc3ccafcc51b66303c2c263aa38486.jpg,33.550304
4,8f49844c382931444e68dffbe20228f4.jpg,33.225216
5,b03f7041962238a7c9d6537e22f9b017.jpg,34.520641
6,c978013571258ed6d4637f6e8cc9d6a3.jpg,35.106148
7,e0de453c1bffc20c22b072b34b54e50f.jpg,37.305515
